In [236]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import csv

In [237]:
# TODO: need to prompt the user to input email
# USER_EMAIL = "lbo@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: Stephanie
USER_EMAIL = "stephanie.francor@gmail.com"
PASSWORD = "wDl37N16v$^a"

# Account: DSIS
# USER_EMAIL = "dsis@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: BHI
# USER_EMAIL = "BHI2@multusmanagement.com"
# PASSWORD = "5Az*32MjEW4p"

# Account: Alpine
# USER_EMAIL = "alpine@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

# Account: PT300
# USER_EMAIL = "pt300@multusmanagement.com"
# PASSWORD = "B3lieveSt0ne!"

MAIN_PAGE_URL = "https://www.airbnb.com/login"
LISTING_PAGE_URL = "https://www.airbnb.com/hosting/listings"
RESERVATION_URL = "https://www.airbnb.com/hosting/reservations"

XPATH_email_btn = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/div[3]/div/div[4]/button/div/div[2]"
XPATH_login_form = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/div/div/div/form"
XPATH_completed_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/div[4]/div[1]/div/div/div/div/div[1]/div/button[2]"
XPATH_canceled_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/div[4]/div[1]/div/div/div/div/div[1]/div/button[3]"
XPATH_reservation_next_page_btn = "/html/body/div[5]/div/div/div[1]/div/div/main/div[1]/section/footer/div/nav/div/button[8]"
CSS_SELECTOR_next_btn = "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"
                        # "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"
                        # "#site-content > div._1u2ubxs > section > footer > div > nav > div > button._1bfat5l.l1j9v1wn.dir.dir-ltr"


In [238]:
driver = webdriver.Chrome()
driver.get(MAIN_PAGE_URL)

In [239]:
def login(driver):
    # click continue with email with XPATH - doable
    email_btn = driver.find_element(By.XPATH, XPATH_email_btn)
    email_btn.click() 

    # try input user info - workable
    email_input = driver.find_element(By.NAME, "user[email]")
    email_input.send_keys(USER_EMAIL)

    # find "continue" button and click to submit email
    login_form = driver.find_element(By.XPATH, XPATH_login_form)
    login_form.submit()
    # TODO: after submitting, not shown (might need to sleep)
    time.sleep(5)

    # find password input form and input password
    pw_input = driver.find_element(By.NAME, "user[password]")
    pw_input.send_keys(PASSWORD)
    pw_input.submit()

    # TODO: SUCCESSFULLY LOGGED IN
    time.sleep(5)


In [240]:
login(driver)

In [241]:
driver.get(RESERVATION_URL)

In [242]:
def extract_all_entries(html_source):
    all_entries = html_source.split("<tr><td class=\"_qjegjiv\">")[1:]
    all_entries[0] = "<tr><td class=\"_qjegjiv\">" + all_entries[0]
    all_entries[-1] = all_entries[-1].split("</div></td></tr>")[0]
    return all_entries

def append_one_entry(df, entry):
    splitted_by_status = entry.split("<div class=\"_pgfqnw\"><span>")
    status = splitted_by_status[1].split("</span>")[0]
    splitted_by_others = splitted_by_status[1].split("<td class=\"_qjegjiv\">")
    
    check_in_time = splitted_by_others[1].split("</td>")[0]
    check_out_time = splitted_by_others[2].split("</td>")[0]
    listing_name = splitted_by_others[3].split("</td>")[1].split(">")[1]
    room = listing_name.split("*")[-1]
    confirmation_code = splitted_by_others[4].split("</td>")[0]
    total_payout = splitted_by_others[5].split("</td>")[0].split("$")[1]

    df["listing name"].append(listing_name)
    df["room"].append(room)
    df["confirmation"].append(confirmation_code)
    df["status"].append(status)
    df["check in"].append(check_in_time)
    df["check out"].append(check_out_time)
    df["total payout"].append(total_payout)
    # TODO: internal name
    
def append_one_entry_completed(df, entry):
    splitted_by_status = entry.split("<div class=\"_pgfqnw\"><span")
    status = splitted_by_status[1].split("</span>")[0][18:]
    splitted_by_others = splitted_by_status[1].split("<td class=\"_qjegjiv\">")
    
    check_in_time = splitted_by_others[1].split("</td>")[0]
    check_out_time = splitted_by_others[2].split("</td>")[0]
    listing_name = splitted_by_others[3].split("</td>")[1].split(">")[1]
    room = listing_name.split("*")[-1]
    confirmation_code = splitted_by_others[4].split("</td>")[0]
    total_payout = splitted_by_others[5].split("</td>")[0].split("$")[1]

    df["listing name"].append(listing_name)
    df["room"].append(room)
    df["confirmation"].append(confirmation_code)
    df["status"].append(status)
    df["check in"].append(check_in_time)
    df["check out"].append(check_out_time)
    df["total payout"].append(total_payout)
    # TODO: internal name
    
def append_all_entries(df, html_source):
    all_entries = extract_all_entries(html_source)
    for entry in all_entries:
        # print(entry)
        append_one_entry(df, entry)

def append_all_entries_completed(df, html_source):
    all_entries = extract_all_entries(html_source)
    for entry in all_entries:
        # print(entry)
        append_one_entry_completed(df, entry)
    
def append_all_pages_upcoming(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            print("no next page")
            # break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(3.5)

def append_all_pages_completed(driver, df):
    html_source = ""
    # counter = 0
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries_completed(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            print("no next page")
            # break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(5)
        # counter = counter + 1

def append_all_pages_canceled_test(driver, df):
    html_source = ""
    while (html_source != driver.page_source):
        html_source = driver.page_source
        append_all_entries(df, html_source)
        # next_page_btn = driver.find_element(By.XPATH, XPATH_reservation_next_page_btn)
        potential_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR_next_btn)
        if len(potential_elements) == 0:
            break
        else:
            next_page_btn = potential_elements[0]
            next_page_btn.click()
        time.sleep(4)



In [243]:
df_dict_upcoming = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}
# append_all_entries_completed(df_dict_upcoming, driver.page_source)
append_all_pages_upcoming(driver, df_dict_upcoming)
df_upcoming = pd.DataFrame(df_dict_upcoming)
df_upcoming

,listing name,room,confirmation,status,check in,check out,total payout
0,Sleep 5| Pet&amp;Baby| Office with Door| Laund...,Sleep 5| Pet&amp;Baby| Office with Door| Laund...,HM5Y34CZTJ,Currently hosting,"Dec 10, 2022","Mar 25, 2023","6,833.84"
1,Sleep6*King*Renovated*PrivateBackyard*Downtown...,L38,HMK8FQ5XHS,Currently hosting,"Feb 26, 2023","Apr 7, 2023",0.00
2,Luxury*HighRise w/ PrivateBalcony*BestLocation...,22A,HMCNEA2QNH,Currently hosting,"Feb 26, 2023","Apr 22, 2023","3,953.47"
3,Sleeps8* Luxury* Backyard* NewlyRenovated* L78,L78,HMFXQEYMD2,Currently hosting,"Feb 28, 2023","Mar 31, 2023","2,578.91"
4,2 King Beds*Sleeps 8*Pool Table*Laundry*Luxury...,Luxury hdh,HM4Q4S9NJ5,Currently hosting,"Mar 3, 2023","Apr 8, 2023","4,463.51"
...,...,...,...,...,...,...,...
235,Sleep 16*6BR3.5BA*FencedYard*Baby*Laundry*WIFI...,WIFI ste,HM84RJH2HT,Confirmed,"Dec 1, 2023","Dec 3, 2023",776.00
236,2 King Beds*Sleeps 8*Pool Table*Laundry*Luxury...,Luxury hdh,HMDDZBSRR5,Confirmed,"Dec 22, 2023","Dec 29, 2023","1,863.25"
237,Sleep 16*6BR3.5BA*FencedYard*Baby*Laundry*WIFI...,WIFI ste,HMMSKKKQZW,Confirmed,"Apr 6, 2024","Apr 8, 2024",698.40
238,Sleep 16*6BR3.5BA*FencedYard*Baby*Laundry*WIFI...,WIFI ste,HM548FWM2W,Confirmed,"May 2, 2024","May 6, 2024","1,122.10"


In [244]:
df_upcoming.to_csv("./03-20-reservations/stephanie-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./DSIS-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./BHI-reservation-upcoming.csv", index=False)
# df_upcoming.to_csv("./PT300-reservation-upcoming.csv", index=False)

In [101]:
# df.to_csv("./out.csv", index=False)
# df["xxxx"] == "arriving ......"

In [245]:
# getting completed reservations
completed_bnt = driver.find_element(By.XPATH, XPATH_completed_btn)
completed_bnt.click()

In [246]:
df_dict_completed = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}

append_all_pages_completed(driver, df_dict_completed)
df_completed = pd.DataFrame(df_dict_completed)
df_completed

,listing name,room,confirmation,status,check in,check out,total payout
0,BeachFront*2 Full*WIFI*Security*Laundry*Room-126,Room-126,HM3ZE5YANM,Review guest,"Mar 18, 2023","Mar 19, 2023",126.10
1,BeachFront* Scooter* WIFI* Laundry* Room-119,Room-119,HM4WFE9FBS,Review guest,"Mar 18, 2023","Mar 19, 2023",116.40
2,Beach Front*Sleep4*WIFI*Security*Laundry*Room-224,Room-224,HMSEZ93XYW,Review guest,"Mar 18, 2023","Mar 19, 2023",116.40
3,BeachFront*2 Full*WIFI*Security*Laundry*Room-131,Room-131,HMB5PCZBFR,Review guest,"Mar 18, 2023","Mar 19, 2023",126.10
4,*Beachfront*Scooter*Sleep5*KingBed*Grill*Room-112,Room-112,HME345MZE8,Review guest,"Mar 18, 2023","Mar 19, 2023",153.26
...,...,...,...,...,...,...,...
3540,Discard: Anna,Discard: Anna,HMR2K3N9TY,Past guest,"May 26, 2017","May 29, 2017",369.00
3541,Discard: Anna,Discard: Anna,HMMCKTRKEA,Past guest,"May 19, 2017","May 21, 2017",262.00
3542,Discard: Anna,Discard: Anna,HMXWPANNF2,Past guest,"Mar 31, 2017","Apr 3, 2017",369.00
3543,Discard: Anna,Discard: Anna,HMQHP34CY9,Past guest,"Mar 21, 2017","Mar 24, 2017",369.00


In [247]:
df_completed.to_csv("./03-20-reservations/stephanie-reservation-completed.csv", index=False)
# df_completed.to_csv("./DSIS-reservation-completed_new.csv", index=False)
# df_completed.to_csv("./BHI-reservation-completed.csv", index=False)
# df_completed.to_csv("./PT300-reservation-completed.csv", index=False)

In [248]:
# getting canceled reservations
canceled_bnt = driver.find_element(By.XPATH, XPATH_canceled_btn)
canceled_bnt.click()

In [249]:
df_dict_canceled = {
    "listing name": [],
    "room": [],
    "confirmation": [],
    "status": [],
    "check in": [],
    "check out": [],
    "total payout": []
}

append_all_pages_canceled_test(driver, df_dict_canceled)
df_canceled = pd.DataFrame(df_dict_canceled)
df_canceled

,listing name,room,confirmation,status,check in,check out,total payout
0,Discard: 2635 Leafmore,Discard: 2635 Leafmore,HMSTDSX8WX,Canceled by guest,"Jul 27, 2023","Aug 2, 2023",0.00
1,Sleep 7| King Bed| Fireplace| TruistPark| lfu,Sleep 7| King Bed| Fireplace| TruistPark| lfu,HMDF9C42SB,Canceled by guest,"Jun 9, 2023","Jun 12, 2023",0.00
2,Discard: 172 bke,Discard: 172 bke,HM3NC3NC3Y,Canceled by guest,"Jun 1, 2023","Jun 5, 2023",0.00
3,Discard: 2407 cp3,Discard: 2407 cp3,HMWQJ8AXK5,Canceled by Airbnb,"May 27, 2023","May 29, 2023",0.00
4,Sleeps 5|Pet&amp;Baby|Laundry In-Unit| cta,Sleeps 5|Pet&amp;Baby|Laundry In-Unit| cta,HMA549SQDX,Canceled by guest,"May 27, 2023","May 29, 2023",0.00
...,...,...,...,...,...,...,...
697,Discard: Elsa,Discard: Elsa,HMYCCCZCMA,Canceled by Airbnb,"Jan 11, 2019","Jan 13, 2019",0.00
698,Discard: Elsa,Discard: Elsa,HMRZ2KSHWW,Canceled by guest,"Nov 10, 2018","Nov 12, 2018",0.00
699,Discard: Anna,Discard: Anna,HMTHQMXADC,Canceled by guest,"Apr 6, 2018","Apr 8, 2018",160.05
700,Discard: Anna,Discard: Anna,HME48ETYAZ,Canceled by guest,"Nov 10, 2017","Nov 12, 2017",0.00


In [250]:
df_canceled.to_csv("./03-20-reservations/stephanie-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./DSIS-reservation-canceled_new.csv", index=False)
# df_canceled.to_csv("./BHI-reservation-canceled.csv", index=False)
# df_canceled.to_csv("./PT300-reservation-canceled.csv", index=False)

In [ ]:
# internal name XPATH
# /html/body/div[6]/div/div/div[1]/div/div/main/div/div/div/div/section[1]/div/div/div[2]/div[1]/table/tbody[2]/tr[1]/td[2]/div/div[2]/div[2]